In [1]:
import argparse

import torch
from torch.utils.data import DataLoader

from kg_model import KG_model

from ogb.linkproppred import Evaluator, PygLinkPropPredDataset

from pykeen.evaluation import RankBasedEvaluator
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline 

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = PygLinkPropPredDataset(name='ogbl-ddi')
data = dataset[0]
data

Data(num_nodes=4267, edge_index=[2, 2135822])

In [12]:
data.edge_index

tensor([[4039, 2424, 4039,  ...,  338,  835, 3554],
        [2424, 4039,  225,  ...,  708, 3554,  835]])

In [4]:
split_edge = dataset.get_edge_split()
train_edge, valid_edge, test_edge = split_edge["train"], split_edge["valid"], split_edge["test"]
train_edge

{'edge': tensor([[4039, 2424],
         [4039,  225],
         [4039, 3901],
         ...,
         [ 647,  708],
         [ 708,  338],
         [ 835, 3554]])}

In [45]:
def convert_to_triples_factory(data):
    tf_data = TriplesFactory.from_labeled_triples(
        data[["head", "relation", "tail"]].values,
        create_inverse_triples=True,
        entity_to_id=None,
        relation_to_id=None,
        compact_id=False 
    )

    print(tf_data.mapped_triples)

    return tf_data

In [46]:
import pandas as pd

# add relation type - interacts with


train = train_edge['edge']
train = torch.tensor([[x[0], 0, x[1]] for x in train])
train_df = pd.DataFrame(train, columns=['head', 'relation', 'tail']).astype(str)

valid = valid_edge['edge']
valid = torch.tensor([[x[0], 0, x[1]] for x in valid])
valid_df = pd.DataFrame(valid, columns=['head', 'relation', 'tail']).astype(str)

test = test_edge['edge']
test = torch.tensor([[x[0], 0, x[1]] for x in test])
test_df = pd.DataFrame(test, columns=['head', 'relation', 'tail']).astype(str)

train_tf = convert_to_triples_factory(train_df)
valid_tf = convert_to_triples_factory(valid_df)
test_tf = convert_to_triples_factory(test_df)

tensor([[   0,    0,  667],
        [   0,    0, 1182],
        [   0,    0, 1280],
        ...,
        [4266,    0, 4250],
        [4266,    0, 4252],
        [4266,    0, 4260]])
tensor([[   0,    0,  729],
        [   1,    0,  681],
        [   1,    0,  768],
        ...,
        [3812,    0, 3722],
        [3812,    0, 3758],
        [3812,    0, 3802]])
tensor([[   0,    0,    3],
        [   0,    0,  185],
        [   0,    0,  187],
        ...,
        [1611,    0, 1562],
        [1611,    0, 1573],
        [1611,    0, 1601]])


In [47]:
train_tf

TriplesFactory(num_entities=4267, num_relations=2, create_inverse_triples=True, num_triples=1067911)

In [48]:
model = KG_model('TransE', train_tf, valid_tf, test_tf, 'ogb')
model.set_params(5, 'Adam', RankBasedEvaluator, 'gpu')
print('Training...')
model.train()
print('Training done')

No random seed is specified. Setting to 2584837398.


Training...


INFO:pykeen.training.training_loop:=> no checkpoint found at 'kg_checkpoints/TransE-ogb_checkpoint.pt'. Creating a new file.
INFO:pykeen.triples.triples_factory:Creating inverse triples.
Training epochs on cuda:0:   0%|                       | 0/5 [00:00<?, ?epoch/s]INFO:pykeen.triples.triples_factory:Creating inverse triples.

Training batches on cuda:0:  60%|████▏  | 4996/8344 [00:09<00:06, 537.48batch/s]


Training batches on cuda:0: 100%|██████▉| 8306/8344 [00:15<00:00, 552.93batch/s]
Training epochs on cuda:0:  20%|▏| 1/5 [00:15<01:03, 15.80s/epoch, loss=0.55, pr
Training batches on cuda:0:  22%|█▌     | 1867/8344 [00:03<00:12, 533.51batch/s]


Training batches on cuda:0:  87%|██████ | 7253/8344 [00:13<00:01, 546.36batch/s]


...

Training batches on cuda:0:  74%|█████▏ | 6140/8344 [00:11<00:04, 523.86batch/s]


Training batches on cuda:0:  99%|██████▉| 8296/8344 [00:15<00:00, 510.90batch/s]
Training epochs on cuda:0:  80%|▊| 4/5 [01:03<00:15, 15.98s/epoch, loss=0.234, p
Training batches on cuda:0:  34%|██▍    | 2847/8344 [00:05<00:10, 528.72batch/s]


Training batches on cuda:0:  96%|██████▋| 7992/8344 [00:15<00:00, 520.95batch/s]


Training batches on cuda:0: 100%|██████▉| 8310/8344 [00:15<00:00, 519.72batch/s]
Training epochs on cuda:0:  80%|▊| 4/5 [01:19<00:15, 15.98s/epoch, loss=0.229, pINFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 5.
Training epochs on cuda:0: 100%|█| 5/5 [01:19<00:00, 15.99s/epoch, loss=0.229, p
INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=1024.
Evaluating on cuda:0: 100%|█████████████| 133k/133k [00:41<00:00, 3.22ktriple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 44.67s seconds


Training done


In [52]:
from ogb.linkproppred import Evaluator

evaluator = Evaluator(name = 'ogbl-ddi')
# You can learn the input and output format specification of the evaluator as follows.
print(evaluator.expected_input_format) 
# print(evaluator.expected_output_format) 

==== Expected input format of Evaluator for ogbl-ddi
{'y_pred_pos': y_pred_pos, 'y_pred_neg': y_pred_neg}
- y_pred_pos: numpy ndarray or torch tensor of shape (num_edges, ). Torch tensor on GPU is recommended for efficiency.
- y_pred_neg: numpy ndarray or torch tensor of shape (num_edges, ). Torch tensor on GPU is recommended for efficiency.
y_pred_pos is the predicted scores for positive edges.
y_pred_neg is the predicted scores for negative edges.
Note: As the evaluation metric is ranking-based, the predicted scores need to be different for different edges.


In [ ]:
# pipeline_result = pipeline(
#     dataset='BioKG',
#     model='TransE',
#     epochs=5,
# )

In [50]:
pipeline_result.get_metric('MRR')

AttributeError: 'KG_model' object has no attribute 'result'

In [12]:
pipeline_result.metric_results.to_df()

,Side,Type,Metric,Value
0,head,optimistic,z_arithmetic_mean_rank,763.334352
1,tail,optimistic,z_arithmetic_mean_rank,774.100216
2,both,optimistic,z_arithmetic_mean_rank,1087.127262
3,head,realistic,z_arithmetic_mean_rank,763.334343
4,tail,realistic,z_arithmetic_mean_rank,774.100210
5,both,realistic,z_arithmetic_mean_rank,1087.127251
6,head,pessimistic,z_arithmetic_mean_rank,763.334327
7,tail,pessimistic,z_arithmetic_mean_rank,774.100202
8,both,pessimistic,z_arithmetic_mean_rank,1087.127234
9,head,optimistic,standard_deviation,6107.172591


In [3]:
import pandas as pd
import json

ddi = pd.read_csv("data/triplets/ddi.tsv", sep='\t', index_col=[0])
drug_ingredients = pd.read_csv("data/triplets/ingredients.tsv", sep='\t', index_col=[0])
food_compounds = pd.read_csv("data/triplets/food_compound.tsv", sep='\t', index_col=[0])

In [4]:
ddi.head()

,drug1,interaction,drug2
0,Apixaban,increase_anticoagulant_activities,Lepirudin
1,Dabigatran etexilate,increase_anticoagulant_activities,Lepirudin
2,Dasatinib,increase_bleeding,Lepirudin
3,Dasatinib,increase_hemorrhage,Lepirudin
4,Deferasirox,increase_gastrointestinal_bleeding,Lepirudin


In [5]:
compounds = set(food_compounds.compound)

food_compound_dict = dict()

for c in compounds:
    food_compound_dict[c] = list(food_compounds[food_compounds['compound'] == c].food)

food_compound_dict['Zinc'][:10]    

['Garden onion',
 'Mugwort',
 'Roman camomile',
 'Ceylon cinnamon',
 'Common hazelnut',
 'Wild carrot',
 'Cornmint',
 'Evening primrose',
 'Scarlet bean',
 'Cloudberry']

In [6]:
drug_compound_dict = dict()
drugs = set(drug_ingredients.drug_name)
interacting_drugs = set(ddi.drug1)
interacting_drugs.union(set(ddi.drug2))

for d in drugs:
    if d in interacting_drugs:
        drug_compound_dict[d] = list(drug_ingredients[drug_ingredients['drug_name'] == d].ingredient)

drug_compound_dict['Galantamine'][:10]  

['Galantamine']

In [6]:
with open('data/drug_compound_dict.json', 'w') as f:
    f.write(json.dumps(drug_compound_dict))

In [7]:
import itertools 

ingredients = list(drug_compound_dict.values())
ingredients = list(itertools.chain.from_iterable(ingredients))
ingredients = set([x.strip() for x in ingredients])
print('Number of interacting drugs ingredients:', len(ingredients))

print('Number of food compounds:', len(food_compound_dict.keys()))
ingredients.intersection(food_compound_dict.keys())

Number of interacting drugs ingredients: 758
Number of food compounds: 276


{'Adenosine',
 'Allantoin',
 'Ascorbic acid',
 'Atropine',
 'Caffeine',
 'Calcium',
 'Capsaicin',
 'Chromium',
 'Diazepam',
 'Dopamine',
 'Ethanol',
 'Formaldehyde',
 'Glycine',
 'Iron',
 'Melatonin',
 'Nicotinamide',
 'Nicotine',
 'Phenol',
 'Progesterone',
 'Pyridoxine',
 'Selenium',
 'Testosterone',
 'Theophylline',
 'Thiamine',
 'Vitamin D'}

In [19]:
from collections import Counter

def get_similar_food(drug_ingredients, food_compound_dict):
    sim_food = []

    for ingredient in drug_ingredients:
        # print(ingredient)
        foods = food_compound_dict.get(ingredient)
        if foods is not None:
            sim_food += foods    

    if len(sim_food) < 1:
        return []

    counts = Counter(sim_food)
    counts = dict(sorted(counts.items(), key=lambda item: item[1]))
    print(counts)
    return sim_food    

In [20]:
# Find a food with the most overlapping content with the interacting drug:

stop = 0
inspected_drugs = []

for row in ddi.itertuples():
    drug1 = row[1]
    effect = row[2]
    drug2 = row[3]

    if drug2 in inspected_drugs:
        continue

    inspected_drugs.append(drug2)
  
    drug2_ingredients = drug_compound_dict.get(drug2)
    
    if drug2_ingredients is not None:
        sim_food = get_similar_food(drug2_ingredients, food_compound_dict)

        
        

{'Savoy cabbage': 1, 'Kiwi': 1, 'Allium': 1, 'Garden onion': 1, 'Leek': 1, 'Garlic': 1, 'Chives': 1, 'Cashew nut': 1, 'Pineapple': 1, 'Dill': 1, 'Wild celery': 1, 'Peanut': 1, 'Burdock': 1, 'Horseradish': 1, 'Tarragon': 1, 'Asparagus': 1, 'Oat': 1, 'Star fruit': 1, 'Brazil nut': 1, 'Common beet': 1, 'Borage': 1, 'Swede': 1, 'Rape': 1, 'Common cabbage': 1, 'Cauliflower': 1, 'Brussel sprouts': 1, 'Broccoli': 1, 'Chinese cabbage': 1, 'Turnip': 1, 'Pigeon pea': 1, 'Tea': 1, 'Pepper': 1, 'Papaya': 1, 'Safflower': 1, 'Caraway': 1, 'Pecan nut': 1, 'Chestnut': 1, 'Chickpea': 1, 'Endive': 1, 'Chicory': 1, 'Chinese cinnamon': 1, 'Watermelon': 1, 'Lemon': 1, 'Mandarin orange (Clementine, Tangerine)': 1, 'Sweet orange': 1, 'Coffee': 1, 'Arabica coffee': 1, 'Robusta coffee': 1, 'Coriander': 1, 'Saffron': 1, 'Muskmelon': 1, 'Cucumber': 1, 'Cucurbita': 1, 'Wild carrot': 1, 'Japanese persimmon': 1, 'Loquat': 1, 'Common buckwheat': 1, 'Fig': 1, 'Fennel': 1, 'Strawberry': 1, 'Soy bean': 1, 'Sunflower': 